In [45]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [46]:
import os
import shutil
from fastai.vision import *
from fastai.metrics import error_rate
from pathlib import Path

In [ ]:


def extract_all(archives, extract_path):
    for filename in archives:
        shutil.unpack_archive(filename, extract_path)

In [47]:

root_dir = os.path.dirname(os.path.dirname(os.path.abspath("__file__")))
data_folder = os.path.join(root_dir, "data")
carparking_dataset_path = os.path.join(root_dir, "data","PKLot.tar.gz")

In [11]:
# unpack dataset
shutil.unpack_archive(carparking_dataset_path, data_folder)

In [29]:
# fix folder structure 
path_to_data="C:/Users/sheec/Desktop/Projects/govhack2019/data/PKLot/PKLotSegmented"
path_to_data_occupied=os.path.join(path_to_data,"Occupied")
path_to_data_empty=os.path.join(path_to_data,"Empty")
for subfolder in os.listdir(path_to_data):
    if subfolder not in ["Occupied", "Empty"]:
        subfolderOccupied = os.path.join(path_to_data, subfolder, "Occupied")
        subfolderEmpty = os.path.join(path_to_data, subfolder, "Empty")
        if os.path.isdir(subfolderOccupied):
            for Occupiedfile in os.listdir(subfolderOccupied):
                shutil.move(os.path.join(subfolderOccupied, Occupiedfile), os.path.join(path_to_data_occupied,Occupiedfile))
        if os.path.isdir(subfolderEmpty):
            for Emptyfile in os.listdir(subfolderEmpty):
                shutil.move(os.path.join(subfolderEmpty, Emptyfile), os.path.join(path_to_data_empty,Emptyfile))
            


In [48]:
# setting system settings 
bs = 64 # batch size
path_img = Path(path_to_data)
path_img

WindowsPath('C:/Users/sheec/Desktop/Projects/govhack2019/data/PKLot/PKLotSegmented')

In [49]:
data = ImageDataBunch.from_folder(path_img, ds_tfms=get_transforms(), size=299, bs=bs, valid_pct=0.2).normalize(imagenet_stats)

C:\ProgramData\Anaconda3\lib\site-packages\fastai\data_block.py:451: UserWarning: Your training set is empty. If this is by design, pass `ignore_empty=True` to remove this warning.
  warn("Your training set is empty. If this is by design, pass `ignore_empty=True` to remove this warning.")
C:\ProgramData\Anaconda3\lib\site-packages\fastai\data_block.py:454: UserWarning: Your validation set is empty. If this is by design, use `split_none()`
                 or pass `ignore_empty=True` when labelling to remove this warning.
  or pass `ignore_empty=True` when labelling to remove this warning.""")


IndexError: index 0 is out of bounds for axis 0 with size 0

In [41]:
print(data.classes)
len(data.classes),data.c

['Empty', 'Occupied']


(2, 2)

In [42]:
learn = cnn_learner(data, models.resnet50, metrics=error_rate)

In [43]:
learn.model

Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Conv2d(64, 256

In [ ]:
learn.fit_one_cycle(1)

In [ ]:
learn.save('stage-1')

In [ ]:
# Results
interp = ClassificationInterpretation.from_learner(learn)

losses,idxs = interp.top_losses()

len(data.valid_ds)==len(losses)==len(idxs)

In [ ]:
interp.plot_top_losses(9, figsize=(15,11))

In [ ]:
interp.most_confused(min_val=2)